In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(3)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1794,tt0367959,50000000,82169884,Hannibal Rising,Gaspard Ulliel|Aaran Thomas|Gong Li|Dominic We...,Peter Webber,It started with revenge.,"The story of the early, murderous roots of the...",121,Crime|Drama|Thriller,Ingenious Film Partners|ETIC Films|Quinta Comm...,2/6/2007,6.0,2007
91,tt2938956,25000000,72629670,The Transporter Refueled,Ed Skrein|Ray Stevenson|Loan Chabanol|Gabriell...,Camille Delamarre,He Delivers.,The fast-paced action movie is again set in th...,96,Thriller|Action|Crime,EuropaCorp,9/3/2015,5.3,2015
1787,tt0462396,67000000,25303038,The Last Legion,Colin Firth|Ben Kingsley|Aishwarya Rai Bachcha...,Doug Lefler,The end of an empire...the beginning of a legend.,"As the Roman empire crumbles, young Romulus Au...",102,Action|Adventure|Fantasy|War,Ingenious Film Partners|Quinta Communications|...,4/19/2007,4.9,2007


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

#Расчет доходности фильма - добавляем в датасет серию profit
data['profit'] = data['revenue'] - data['budget']

In [5]:
#Преобразование серии genres для дальнейшей работы
genres = data.copy()
genres.genres = genres.genres.apply(lambda x: x.split('|'))
genres = genres.explode('genres')
genres['profit'] = genres['revenue'] - genres['budget']

In [6]:
#Выбор из исходного датасета data значений, у которых в серии release_year указан 2012 г. В него включаются серии cast и revenue
max_profit = data[data.release_year == 2012][['cast','revenue']]
#Изменение серии cast нового датасета
max_profit.cast = max_profit.cast.apply(lambda y: y.split('|'))
max_profit = max_profit.explode('cast')

In [8]:
#Формирование нового датасета high_budget из исходного, путём включения тех фильмов, значение параметра budget у которых больше среднего
high_budget = data[data.budget > data.budget.mean()]
#Изменение серии cast нового датасета
high_budget.cast = high_budget.cast.apply(lambda z: z.split('|'))
high_budget = high_budget.explode('cast')

In [9]:
#Преобразование серии production_companies для дальнейшей работы
studios = data.copy()
studios.production_companies = studios.production_companies.apply(lambda b: b.split('|'))
studios = studios.explode('production_companies')
studios['profit'] = studios['revenue'] - studios['budget']

In [10]:
#Преобразование серии director для дальнейшей работы
directors = data.copy()
directors.director = directors.director.apply(lambda c: c.split('|'))
directors = directors.explode('director')

In [11]:
#Преобразование серии releases для дальнейшей работы
releases = data.copy()
releases.release_date = releases.release_date.apply(lambda d: d.split('/'))

#Удаление из нового датасета releases строк, включающих в себя день и год релиза
releases = releases.explode('release_date')[::3]

#Переименование столбцов в новом датасете
releases.columns=['imdb_id','budget','revenue','original_title','cast','director','tagline','overview','runtime','genres','production_companies','release_month','vote_average','release_year','profit']

In [12]:
#Создание нового датасета length, включающего в себя серию length (количество символов значений парамента original_title)
length = data.copy()
length['length_title'] = length['original_title'].map(lambda e: len(e))
length['length_overview'] = length['overview'].map(lambda f: len(f.split()))
length.production_companies = length.production_companies.apply(lambda g: g.split('|'))
length = length.explode('production_companies')

# 1. У какого фильма из списка самый большой бюджет?

In [13]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [14]:
#Группировка датасета по наименованию/id фильма и поиск максимального значения параметра budget
data.groupby(['original_title','imdb_id'])['budget'].max().sort_values(ascending=False).head(3)

original_title                               imdb_id  
Pirates of the Caribbean: On Stranger Tides  tt1298650    380000000
Pirates of the Caribbean: At World's End     tt0449088    300000000
Avengers: Age of Ultron                      tt2395427    280000000
Name: budget, dtype: int64

# 2. Какой из фильмов самый длительный (в минутах)?

In [15]:
answers['2'] = 'Gods and Generals (tt0279111)'

In [16]:
#Группировка датасета по наименованию/id фильма и поиск максимального значения параметра runtime
data.groupby(['original_title','imdb_id'])['runtime'].max().sort_values(ascending=False).head(3)

original_title                                 imdb_id  
Gods and Generals                              tt0279111    214
The Lord of the Rings: The Return of the King  tt0167260    201
Grindhouse                                     tt0462322    191
Name: runtime, dtype: int64

# 3. Какой из фильмов самый короткий (в минутах)?





In [17]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [18]:
#Группировка датасета по наименованию/id фильма и поиск минимального значения параметра runtime
data.groupby(['original_title','imdb_id'])['runtime'].min().sort_values(ascending=True).head(3)

original_title        imdb_id  
Winnie the Pooh       tt1449283    63
The Jungle Book 2     tt0283426    72
Return to Never Land  tt0280030    72
Name: runtime, dtype: int64

# 4. Какова средняя длительность фильмов?


In [19]:
answers['4'] = 110

In [20]:
#Расчет и округление среднего значения длительности (параметр runtime) всех фильмов в исходном датасете
round(data['runtime'].mean())

110

# 5. Каково медианное значение длительности фильмов? 

In [21]:
answers['5'] = 107

In [22]:
#Расчет и округление медианного значения длительности (параметр runtime) всех фильмов в исходном датасете
round(data['runtime'].median())

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [23]:
answers['6'] = 'Avatar (tt0499549)'

In [24]:
#Группировка датасета по наименованию/id фильма и поиск максимального значения параметра profit
data.groupby(['original_title','imdb_id'])['profit'].max().sort_values(ascending=False).head(3)

original_title                imdb_id  
Avatar                        tt0499549    2544505847
Star Wars: The Force Awakens  tt2488496    1868178225
Jurassic World                tt0369610    1363528810
Name: profit, dtype: int64

# 7. Какой фильм самый убыточный? 

In [25]:
answers['7'] = 'The Lone Ranger (tt1210819)'

In [26]:
#Группировка датасета по наименованию/id фильма и поиск минимального значения параметра profit
data.groupby(['original_title','imdb_id'])['profit'].min().sort_values(ascending=True).head(3)

original_title   imdb_id  
The Lone Ranger  tt1210819   -165710090
The Alamo        tt0318974   -119180039
Mars Needs Moms  tt1305591   -111007242
Name: profit, dtype: int64

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [27]:
answers['8'] = 1478

In [28]:
#Подсчет количества записей в датасете, у которых значение profit больше нуля
s = data[data.profit > 0]['profit'].count()
print(s)

1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
answers['9'] = 'The Dark Knight (tt0468569)'

In [30]:
#Выбор из исходного датасета data значений, у которых в серии release_year указан 2008 год
release_2008 = data[data.release_year == 2008]

#Группировка нового датасета release_2008 по названию/id фильма и выбор максимального значения параметра profit
release_2008.groupby(['original_title','imdb_id'])['profit'].max().sort_values(ascending=False).head(3)

original_title                                      imdb_id  
The Dark Knight                                     tt0468569    816921825
Indiana Jones and the Kingdom of the Crystal Skull  tt0367882    601636033
Mamma Mia!                                          tt0795421    557841637
Name: profit, dtype: int64

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [31]:
answers['10'] = 'The Lone Ranger (tt1210819)'

In [32]:
#Выбор из исходного датасета data значений, у которых в серии release_year указаны 2012-2014 гг.
release_2012_2014 = data.loc[data['release_year'].isin([2012,2013,2014])]

#Группировка нового датасета release_2012_2014 по названию/id фильма и поиск минимального значения параметра profit
release_2012_2014.groupby(['original_title','imdb_id','release_year'])['profit'].min().sort_values(ascending=True).head(3)

original_title   imdb_id    release_year
The Lone Ranger  tt1210819  2013           -165710090
R.I.P.D.         tt0790736  2013            -68351500
Upside Down      tt1374992  2012            -51893525
Name: profit, dtype: int64

# 11. Какого жанра фильмов больше всего?

In [33]:
answers['11'] = 'Drama'

In [34]:
#Выбор наиболее часто встречающегося значения (top) серии genres из сформированного ранее на этапе Предобработки нового датасета genres
genres['genres'].describe(include=['object']).top

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [35]:
answers['12'] = 'Drama'

In [36]:
#Выбор из нового датасета genres всех записей, у которых значение серии profit более нуля
genres[genres.profit > 0]

#Выбор наиболее часто встречающегося значения (top) серии genres из отфильтрованного выше датасета genres
genres['genres'].describe(include=['object']).top

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [37]:
answers['13'] = 'Peter Jackson'

In [38]:
#Группировка исходного датасета по режиссерам с последующей сортировкой сумм кассовых сборов (revenue)
directors.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head(3)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
answers['14'] = 'Robert Rodriguez'

In [40]:
#Группировка в новом датасете genres_action всех записей, параметр genres которых содержит строку 'Action'
genres_action = genres[genres.genres.str.match('Action', na=False)]

#Подсчет количества значений параметра director (режиссер)
genres_action = genres_action['director'].str.split('|', expand = True).stack().value_counts()
display(genres_action.head(3))

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
answers['15'] = 'Chris Hemsworth'

In [42]:
#Группировка измененного на этапе Предобработки датасета max_profit по актерам с последующей сортировкой сумм кассовых сборов (revenue)
max_profit.groupby(['cast'])['revenue'].sum().sort_values(ascending=False).head(3)

cast
Chris Hemsworth    2027450773
Denis Leary        1629460639
Anne Hathaway      1522851057
Name: revenue, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = 'Matt Damon'

In [44]:
#Группировка измененного на этапе Предобработки датасета high_budget по актерам с последующим подсчетом уникальных наименований параметра original_title (т.е. названий фильмов)
high_budget.groupby(['cast'])['original_title'].nunique().sort_values(ascending=False).head(3)

cast
Matt Damon        18
Adam Sandler      17
Angelina Jolie    16
Name: original_title, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
answers['17'] = 'Action'

In [46]:
#Создание нового датасета Cage из исходного data, в который вошли значения строки cast со значением Nicolas Cage
Cage = data[data.cast.str.contains('Nicolas Cage', na = False)]

#Разделение датасета Cage на отдельные строки по каждому значению параметра genres
Cage = Cage.genres.apply(lambda d: d.split('|'))
Cage_genres = Cage.explode()

#Сбор статистики
Cage_genres.describe()

count         93
unique        15
top       Action
freq          17
Name: genres, dtype: object

# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

In [48]:
#Выбор из нового датасета studios записей, содержащих Paramount Pictures
studios = studios[studios.production_companies.str.contains('Paramount Pictures', na = False)]

#Группировка измененного датасета по названию/id фильма с выбором минимального значения параметра profit
studios.groupby(['original_title','imdb_id'])['profit'].min().sort_values(ascending=True).head(3)

original_title        imdb_id  
K-19: The Widowmaker  tt0267626   -64831034
Timeline              tt0300556   -60519261
Next                  tt0435705   -51788987
Name: profit, dtype: int64

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
answers['19'] = '2015'

In [50]:
#Группировка исходного датасета по году выпуска с суммированием значений параметра прибыльность (profit)
data.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(3)

release_year
2015    18668572378
2014    16397812953
2012    16077001687
Name: profit, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
answers['20'] = '2014'

In [52]:
#Создание нового датасета warner из исходного, включающего в себя значения параметра production_companies 'Warner Bros'
warner = data[data.production_companies.str.contains('Warner Bros', na = False)]

#Группировка нового датасета по серии release_year с суммированием по параметру profit
warner.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(3)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
answers['21'] = 'Сентябрь (9)'

In [54]:
#Определение максимального значения по параметру release_month в новом датасете releases
releases.release_month.max()

'9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [55]:
answers['22'] = 450

In [56]:
#Создание нового датасета на основе записей, в которых параметр release_date начинается с 6/ или 7/ или /8 (июнь, июль, август)
summer = data
summer = summer[summer.release_date.str.match('6\D|7\D|8\D', na = False)]

#Подсчет количества записей в новом датасете
summer.original_title.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [57]:
answers['23'] = 'Peter Jackson'

In [58]:
#Создание нового дата сета на основе записей, в которых параметр release_date начинается с 12/ или 1/ или /2 (декабрь, январь, февраль)
winter = data
winter = winter[winter.release_date.str.match('12\D|1\D|2\D', na = False)]

#Группировка нового датасета по режиссеру (director) с подсчетом количества фильмов (параметр original_title)
winter.groupby(['director'])['original_title'].count().sort_values(ascending=False).head(3)

director
Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Name: original_title, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
answers['24'] = 'Four By Two Productions'

In [60]:
#Группировка датасета length по производственной компании (production_companies) с подсчетом среднего количества символов по параметру length_title
length.groupby(['production_companies'])['length_title'].mean().sort_values(ascending=False).head(3)

production_companies
Four By Two Productions    83.0
Jim Henson Company, The    59.0
Dos Corazones              47.0
Name: length_title, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [61]:
answers['25'] = 'Midnight Picture Show'

In [62]:
#Группировка датасета length по производственной компании (production_companies) с подсчетом среднего количества символов по параметру length_overview
length.groupby(['production_companies'])['length_overview'].mean().sort_values(ascending=False).head(3)

production_companies
Midnight Picture Show    175.0
Room 9 Entertainment     161.0
98 MPH Productions       159.0
Name: length_overview, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [63]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

In [64]:
#Создание нового датасета, в который включены те из фильмов, у которых значение параметра vota_average больше, чем у 99% выборки
best = data.loc[data['vote_average'] > np.percentile(data.vote_average,99)]
display(best['original_title'])

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
answers['27'] = 'Daniel Radcliffe, Rupert Grint'

In [66]:
#Создание списка из серии cast начального датасета data
cast = data.cast.str.split('|').tolist()
cast_list = []

#Формирование нового списка cast_list из комбинаций созданного ранее списка cast
for i in range(len(cast)):
    cast_list.append(list(combinations(cast[i],2)))

#Создание датафрейма cast из списка cast_list, включающего в себя все "пары" актеров
cast = pd.DataFrame(cast_list)
cast.describe()

,0,1,2,3,4,5,6,7,8,9
count,1888,1888,1888,1888,1888,1888,1885,1885,1885,1885
unique,1803,1836,1851,1861,1849,1866,1870,1866,1866,1873
top,"(Daniel Radcliffe, Rupert Grint)","(Daniel Radcliffe, Emma Watson)","(Jennifer Lawrence, Woody Harrelson)","(Kristen Stewart, Peter Facinelli)","(Rupert Grint, Emma Watson)","(Josh Hutcherson, Woody Harrelson)","(Ian McKellen, Orlando Bloom)","(Liam Hemsworth, Woody Harrelson)","(Liam Hemsworth, Elizabeth Banks)","(Liv Tyler, Orlando Bloom)"
freq,7,7,4,3,7,4,3,4,3,3


# Submission

In [67]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь (9)',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe, Rupert Grint'}

In [68]:
# и убедиться что ни чего не пропустил)
len(answers)

27